In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [ ]:
from simulator import Simulator, div

In [ ]:
def force(m,n, w):
    f = np.zeros((m,n,2))
    f[5:-5,40:60,0] = 1 - w[5:-5,40:60,0]
    f += np.random.normal(size=(m,n,2))/3
    return f

def border_condition(w):
    w[0,:,0] = 0
    w[-1,:,0] = 0
    w[:,0,1] = 0
    w[:,-1,1] = 0

In [ ]:
m,n = 80,100
sim = Simulator(m,n, 1, 0.01, border_condition, force)

In [ ]:
x = np.zeros((m,n))

def take_scalar(a,indices_i, indices_j) :
    m,n = a.shape[:2]
    full_idices = n*indices_i + indices_j
    reshaped = a.reshape(m*n)
    res = np.take(reshaped,full_idices, axis=0)
    return res.reshape(m,n)

def update_scalar(s,w,dt) :
    m,n = w.shape[:2]
    indices = np.indices((m,n)).swapaxes(0,2).swapaxes(0,1) - dt*w
    indices_floor = np.floor(indices).astype(np.int)
    frac = indices-indices_floor
    frac = [1-frac,frac]
    indices_ceil = indices_floor + 1
    indices_floor[:,:,0] = np.clip(indices_floor[:,:,0], 0, m-1)
    indices_ceil[:,:,0] = np.clip(indices_ceil[:,:,0], 0, m-1)
    indices_floor[:,:,1] = np.clip(indices_floor[:,:,1], 0, n-1)
    indices_ceil[:,:,1] = np.clip(indices_ceil[:,:,1], 0, n-1)
    indices = [indices_floor, indices_ceil] # shape : 2 m,n,2

    res = np.zeros((m,n))
    for i in range(2) :
        for j in range(2) :
            res += np.reshape(frac[i][:,:,0]*frac[j][:,:,1], (m,n)) * take_scalar(x, indices[i][:,:,0], indices[j][:,:,1])
    res = res*1
    res[0:2,40:60] = 1
    return res

for i in range(10000) :
    dt=0.8
    sim.time_step(dt)
    x = update_scalar(x,sim.w,dt)
    if not i % 1 :
        plt.axis('off')
        plt.imshow(x[::-1,:])
        clear_output()
        plt.show()

In [ ]:
for i in range(10000) :
    sim.time_step(0.4)
    if not i % 10 :
        sim.display()
        plt.axis('off')
        clear_output()
        plt.show()